In [1]:
# 报错values解决方案 https://blog.csdn.net/qq_36423517/article/details/104285012
from tensorflow.keras.models import load_model
my_model = load_model('NN_slabel_classification_model.h5')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 数据加载 
df = pd.read_csv('./raw_label_data.csv',index_col=0)
df.head()

,recordid,screwid,tighteningstatus,iio,shake_num,扭矩值,torquearray,anglearray,label,shake_num_0.2_4
0,91403980,_VW321_1-20-12-01-01,OK,1.0,1.0,"8Nm,AD18","0.02,0.05,0.1,0.36,0.7,1.03,1.74,2.03,2.08,2.2...","0,0,1.06,2.24,3.26,4.13,5.4,6.11,6.43,7.6,8.83...",high,1
1,91398407,_VW321_1-20-12-01-01,OK,1.0,1.0,"8Nm,AD18","0,0.11,0.1,0.1,0.1,0.1,0.09,0.09,0.08,0.09,0.0...","0,1.11,2.22,3.25,4.5,5.63,6.87,7.88,8.98,10.11...",high,1
2,91338160,_VW321_1-20-12-01-01,OK,1.0,1.0,"8Nm,AD18","0,0,0.04,0.04,0.04,0.03,0.04,0.04,0.03,0.04,0....","0,0,1.09,2.27,3.39,4.58,5.74,6.95,8.33,9.45,10...",high,1
3,91424115,_VW321_1-20-12-01-02,OK,2.0,0.0,"8Nm,AD18","0.01,0,0.05,0.05,0.06,0.06,0.05,0.04,0.05,0.05...","0,0,1.14,2.3,3.38,4.61,5.81,7.02,8.03,9.03,10....",high,1
4,91257695,_VW321_1-20-12-01-02,OK,2.0,0.0,"8Nm,AD18","0.02,0.02,0.05,0.05,0.04,0.06,0.05,0.06,0.06,0...","0,0,1.08,2.26,3.36,4.42,5.42,6.68,7.7,9.12,10....",high,0


In [3]:
# 选择需要的列
data = df[['recordid', 'shake_num', 'shake_num_0.2_4', 'tighteningstatus','扭矩值','iio', 'label']]   
#数值化
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
le = LabelEncoder()
data['扭矩值'] = le.fit_transform(data['扭矩值'])
data['tighteningstatus'] = le.fit_transform(data['tighteningstatus'])
label_le = LabelEncoder()
data['label'] = label_le.fit_transform(data['label'])
#归一化
mm = MinMaxScaler()
x = data[[ 'shake_num', 'shake_num_0.2_4', 'tighteningstatus','扭矩值','iio',]]  
x = mm.fit_transform(x)
y = np.array(data['label'])
#划分训练集/测试集
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

f:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
f:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
f:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydat

In [4]:
'''获取labelencoder映射'''
def get_integer_mapping(le):
    '''
    Return a dict mapping labels to their integer values
    from an SKlearn LabelEncoder
    le = a fitted SKlearn LabelEncoder
    '''
    res = {}
    for cl in le.classes_:
        res.update({cl:le.transform([cl])[0]})

    return res

In [5]:
#获取label的数值化标签
label_le.classes_

array(['OK', 'high', 'low', 'medium'], dtype=object)

In [6]:
y_pred = my_model.predict_classes(x)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [7]:
y_df = pd.DataFrame(y_pred)
y_df

,0
0,2
1,2
2,2
3,2
4,0
...,...
1127,2
1128,2
1129,2
1130,2


In [8]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y)

0.931095406360424

In [9]:
s = label_le.classes_
dic_label = []
for i in y_pred:
    dic_label.append(s[i])
print(dic_label)

['low', 'low', 'low', 'low', 'OK', 'low', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'low', 'low', 'high', 'low', 'low', 'low', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'medium', 'medium', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'high', 'OK', 'OK', 'OK', 'high', 'OK', 'OK', 'low', 'low', 'low', 'low', 'OK', 'low', 'high', 'high', 'high', 'high', 'high', 'high', 'medium', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'low', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'low', 'medium', 'medium', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'high', 'OK', 'low', 'OK', 'low', 'OK', 'OK', 'low', 'low', 'medium', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'medium', 'OK', 'OK', 'OK', 'low', 'OK', 'low', 'OK', 'OK', 'low', 'low', 'low', 'low', 'OK', 'OK', 'OK', 'OK', 'OK', 'high', 'OK', 'low', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK

In [10]:
len(y_df)

1132

In [11]:
len(df['label'])

1132

In [12]:
dic_label_df = pd.DataFrame(dic_label)
compare = pd.concat([dic_label_df,df['label']],axis =1)
compare

,0,label
0,low,high
1,low,high
2,low,high
3,low,high
4,OK,high
...,...,...
1127,low,low
1128,low,low
1129,low,low
1130,low,low


In [53]:
#显示Dateframe所有列(参数设置为None代表显示所有行，也可以自行设置数字)
pd.set_option('display.max_columns',None)
#显示Dateframe所有行
pd.set_option('display.max_rows',None)
#设置Dataframe数据的显示长度，默认为50
pd.set_option('max_colwidth',200)
#禁止Dateframe自动换行(设置为Flase不自动换行，True反之)
pd.set_option('expand_frame_repr', False)

In [21]:
data_raw = r'F:\python_notebook\00 Torque\torqueData_shakeNum_index.csv'

In [22]:
data_raw = pd.read_csv(data_raw,index_col=0)
pd.set_option('display.max_columns',None) 
data_raw.head()

,recordid,line_id,idcode,screwid,thresholdtorque1,rotationlspeed1,torque1,torque_max1,torque_min1,angle1,angle_max1,angle_min1,thresholdtorque2,rotationlspeed2,torque2,torque_max2,torque_min2,angle2,angle_max2,angle_min2,thresholdtorque3,rotationlspeed3,torque3,torque_max3,torque_min3,angle3,angle_max3,angle_min3,thresholdtorque4,rotationlspeed4,torque4,torque_max4,torque_min4,angle4,angle_max4,angle_min4,thresholdtorque5,rotationlspeed5,torque5,torque_max5,torque_min5,angle5,angle_max5,angle_min5,tighteningstatus,tighteningtime,channel,cycle,operator,iio,torquearray,anglearray,shake_num,装配内容,扭矩值,torque_index,螺栓编号2
0,91411328,1,78202093911572,_VW331_4-20-73-01-02,0.0,50.0,4.98,5.75,4.25,57.8,720.0,0.0,5.0,20.0,10.0,11.5,8.5,33.2,84.0,5.0,10.0,10.0,20.09,23.0,17.0,13.8,63.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OK,2020/9/22 12:58:04,51,1347902,0.0,2,"0.217,0.217,0.186,0.217,0.402,0.402,0.464,0.49...","0.02,0.04,0.06,0.17,0.66,1.42,2.47,3.86,5.51,7...",0,后保横梁,"20Nm,AD18",AD18,_VW331_4-20-73-01-02
1,91254652,1,78202093905410,_VW331_4-20-73-01-02,0.0,50.0,4.98,5.75,4.25,66.9,720.0,0.0,5.0,20.0,10.0,11.5,8.5,26.0,84.0,5.0,10.0,10.0,20.09,23.0,17.0,16.4,63.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OK,2020/9/21 16:04:20,51,1343139,0.0,2,"0.062,0.062,0.062,0.062,0.155,0.155,0.186,0.15...","0.02,0.04,0.06,0.22,0.71,1.4,2.46,3.86,5.62,7....",0,后保横梁,"20Nm,AD18",AD18,_VW331_4-20-73-01-02
2,91240946,1,78202093904697,_VW331_4-20-73-01-02,0.0,50.0,4.98,5.75,4.25,31.2,720.0,0.0,5.0,20.0,10.0,11.5,8.5,42.1,84.0,5.0,10.0,10.0,20.09,23.0,17.0,53.0,63.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OK,2020/9/21 14:54:47,51,1342731,0.0,2,"0,0,0,0.031,0.031,0.217,0.217,0.279,0.31,0.279...","0.02,0.04,0.06,0.2,0.65,1.37,2.47,3.84,5.55,7....",0,后保横梁,"20Nm,AD18",AD18,_VW331_4-20-73-01-02
3,91371864,1,78202093910131,_VW331_4-20-73-01-02,0.0,50.0,4.98,5.75,4.25,56.4,720.0,0.0,5.0,20.0,10.0,11.5,8.5,27.8,84.0,5.0,10.0,10.0,20.09,23.0,17.0,15.4,63.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OK,2020/9/22 9:11:34,51,1346750,0.0,2,"0.062,0.062,0.062,0,0.186,0.186,0.186,0.248,0....","0.02,0.04,0.06,0.2,0.61,1.35,2.46,3.86,5.56,7....",0,后保横梁,"20Nm,AD18",AD18,_VW331_4-20-73-01-02
4,91372438,1,78202093910834,_VW331_4-20-73-01-02,0.0,50.0,4.98,5.75,4.25,71.7,720.0,0.0,5.0,20.0,10.0,11.5,8.5,29.4,84.0,5.0,10.0,10.0,20.09,23.0,17.0,16.4,63.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OK,2020/9/22 9:14:39,51,1346768,0.0,2,"0.031,0.031,0.062,0.062,0.062,0.093,0.062,0.09...","0.02,0.04,0.06,0.16,0.64,1.45,2.51,3.9,5.57,7....",0,后保横梁,"20Nm,AD18",AD18,_VW331_4-20-73-01-02


In [ ]:
# 选择需要的列
data_test = data_raw[['recordid', 'shake_num', 'shake_num_0.2_4', 'tighteningstatus','扭矩值','iio', 'label']]   
#数值化
le = LabelEncoder()
data_test['扭矩值'] = le.fit_transform(data_test['扭矩值'])
data_test['tighteningstatus'] = le.fit_transform(data_test['tighteningstatus'])
label_le = LabelEncoder()
data['label'] = label_le.fit_transform(data['label'])
#归一化
mm = MinMaxScaler()
x = data[[ 'shake_num', 'shake_num_0.2_4', 'tighteningstatus','扭矩值','iio',]]  
x = mm.fit_transform(x)
#划分训练集/测试集
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)